In [4]:
#initialize
#     model
#     environment variables

import sys
sys.path.append('..')
from src.utils.llamaindex_retriever import LlamaIndexRetriever
from langchain.vectorstores import FAISS
from langchain.embeddings.azure_openai import AzureOpenAIEmbeddings
from langchain.embeddings.openai import OpenAIEmbeddings
from dotenv import load_dotenv
import os
from langchain.chat_models import AzureChatOpenAI

def load_env_variables(file_path):
    load_dotenv(file_path)
    print("Environment variables loaded successfully!")

env_file_path = "../.env"
load_env_variables = load_env_variables(env_file_path)
max_tokens = 3500
temperature = 0.1

# embeddings = AzureOpenAIEmbeddings(azure_deployment=azure_deployment, openai_api_version=openai_api_version)
embeddings =  AzureOpenAIEmbeddings(
        deployment=os.getenv("EMB_DEPLOYMENT"),
        openai_api_version=os.getenv("EMB_OPENAI_API_VERSION"),
        model=os.getenv("EMB_MODEL"),
        openai_api_key=os.getenv("EMB_OPENAI_API_KEY"),
        openai_api_base=os.getenv("EMB_OPENAI_ENDPOINT"),
        openai_api_type=os.getenv("EMB_API_TYPE"),
    )

llm_gpt = AzureChatOpenAI(deployment_name=os.getenv('AZURE_OPENAI_DEPLOYMENT_NAME'), openai_api_version=os.getenv("OPENAI_API_VERSION"),
                        openai_api_base=os.getenv("OPENAI_API_BASE"), 
                        openai_api_type= os.getenv("OPENAI_API_TYPE"),
                        openai_api_key=os.getenv("OPENAI_API_KEY"),
                        max_tokens=max_tokens,
                        temperature=temperature)

Environment variables loaded successfully!


/Users/k.abhishek/anaconda3/envs/agentgpt/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `AzureOpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import AzureOpenAIEmbeddings`.
  warn_deprecated(
/Users/k.abhishek/anaconda3/envs/agentgpt/lib/python3.11/site-packages/langchain_community/embeddings/azure_openai.py:113: UserWarning: As of openai>=1.0.0, Azure endpoints should be specified via the `azure_endpoint` param not `openai_api_base` (or alias `base_url`). Updating `openai_api_base` from https://polarisopenai.openai.azure.com/ to https://polarisopenai.openai.azure.com//openai.
  warnings.warn(
/Users/k.abhishek/anaconda3/envs/agentgpt/lib/python3.11/site-packages/langchain_community/embeddings/azure_openai.py:1

In [22]:
#define utils
from src.utils.cube_semantic_custom import CubeSemanticLoader
def fetch_cube_metadata(*args, **kwargs):
    try:
        # # Load document from Cube meta api
        loader = CubeSemanticLoader(os.getenv("CUBE_API_URL"), os.getenv("CUBE_TOKEN"), False)
        documents = loader.load()
        # to_json()
        return documents
    except Exception as e:
        # Handle exceptions gracefully and return an error response
        print("Error in fetching metadata from cube: " + str(e))
        return 0

def create_vector_store(documents, local_vector_store_path, *args, **kwargs):
    print("Loaded documents: " + str(documents))
    vectorstore = FAISS.from_documents(documents, embeddings)
    vectorstore.save_local(local_vector_store_path)
    print("Vector store created and saved successfully!")

def load_vector_store(vector_store_path, embeddings, *args, **kwargs):
    # Load the vector store from the local file system
    vectorstore = FAISS.load_local(vector_store_path, embeddings, allow_dangerous_deserialization=True)
    print("Vector store loaded successfully!")
    
    return vectorstore

In [23]:
#load existing vector store
vector_store_path = "/Users/k.abhishek/Documents/experiments/metric_store/metric_store_gen_ai/data/vector_store/cube_meta_faiss_index"
vectorstore = load_vector_store(vector_store_path, embeddings)

Vector store loaded successfully!


Tools 

In [30]:
import json
from crewai import Agent, Task, Crew
from langchain.tools import tool
from src.utils.llamaindex_retriever import LlamaIndexRetriever
from typing import Optional, Type
from langchain.callbacks.manager import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)
# Import things that are needed generically
from langchain.pydantic_v1 import BaseModel, Field
# from langchain.tools import BaseTool, StructuredTool, tool
from crewai_tools import BaseTool


def get_similar_documents_faiss(query, max_number_documents=3):
  vectorstore = FAISS.load_local(vector_store_path, embeddings, allow_dangerous_deserialization=True)
  docs = vectorstore.similarity_search_with_relevance_scores(query, max_number_documents)
  relevant_documents = []
  for doc in docs:
      doc = doc[0]
      meta = {'text':doc.page_content, 'table_metadata': doc.metadata}
      relevant_documents.append(meta)
  return relevant_documents

def get_similar_documents(query, max_number_documents=3):
    return get_similar_documents_faiss(query, max_number_documents)


class QueryInput(BaseModel):
    query: str = Field(description="should be enquiry query")


class RephraseInputQuery(BaseTool):
    name:str = "rephrase_input_query"
    description :str = "Useful to rephrase the query to capture the intent of the user regarding metric information"
    args_schema: Type[BaseModel] = QueryInput

    def _run(
        self, query: str, run_manager: Optional[CallbackManagerForToolRun] = None
    ) -> str:
        """Use the tool."""
        metric_description = self.rephrase_input_query(query)
        return metric_description

    async def _arun(
        self, query: str, run_manager: Optional[AsyncCallbackManagerForToolRun] = None
    ) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("custom_search does not support async")
    
    
    def rephrase_input_query(self, query, *args, **kwargs):
        agent = Agent(
                role='Intent Capturer',
                goal=
                'Rephrasing the query to capture the intent of the user regarding metric information',
                backstory=
                "You are an expert to understand the user's intent and rephrase the query to capture the intent of the user accurately.",
                llm = llm_gpt,
                allow_delegation=False)
        task = Task(
                agent=agent,
                description=
                f'Rephrase the query to capture the intent of the user regarding metric information. The query is {query}. Donot add any noise to the response',
                expected_output="some string",
        
            )
        extracted_metrics = task.execute()

        return extracted_metrics
    

    
class MetricDiscovery(BaseTool):
    name :str = "metric_discovery"
    description:str = """Useful for general user questions related to discovery, explaination, description, interpretation of metrics/measures/KPIs, tables or columns."""
    args_schema: Type[BaseModel] = QueryInput

    def _run(
        self, query: str, run_manager: Optional[CallbackManagerForToolRun] = None
    ) -> str:
        """Use the tool."""
        metric_description = self.metric_discovery(query)
        return metric_description

    async def _arun(
        self, query: str, run_manager: Optional[AsyncCallbackManagerForToolRun] = None
    ) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("custom_search does not support async")
    
    
    def metric_discovery(self, query, *args, **kwargs):
        """Useful for general user questions related to discovery, explaination, description, interpretation of metrics/measures/KPIs, tables or columns."""
        relevant_documents = get_similar_documents(query)
        agent = Agent(
                role='Data Analyst Assistant',
                goal=
                'Empower users to understand and utilize data effectively. This includes helping them discover relevant metrics, interpreting their meaning',
                backstory=
                "The primary purpose is to bridge the gap between raw data and user comprehension, fostering a data-driven culture within the organization.",
                llm = llm_gpt,
                allow_delegation=False)  
        
        task = Task(
                agent=agent,
                description=
                """ You are responding to  question {metric_description} with answer from the Metadata provided to you as {relevant_documents}}.
                    Strictly answer the question with the information present in metadata only.
                    Respond with "Sorry, the query is out of scope." if the answer is not present in metadata and terminate further reasoning and prcoess.
                    """,
                expected_output="some string",
        
            )
        output = task.execute()

        return output




Tasks

In [36]:
def check_termination(task_output):
    agent = Agent(
                role='Determine query completion',
                goal=
                'Given task output, determine termination of crew workflow',
                backstory=
                "Given user query request from user determine if no more new task is needed to be executed",
                llm = llm_gpt,
                allow_delegation=False)
    task = Task(
                agent=agent,
                description=
                f'Given task output, if the query is completed and the task result determines that the query is out of scope then terminate the workflow or else the required query is satisfied. Following is the task result {task_output}',
                expected_output="some string",
        
            )
    termination_status = task.execute()
    return termination_status
class MetricDiscoveryTasks():
  def metric_isolation(self, agent, query):
    return Task(description=(f"""
        Rephrase the query to capture the intent of the user regarding metric information. The query is {query}. Donot add any noise to the response.
        {self.__tip_section()}"""),
      agent=agent,
      expected_output="Reformatted query to capture the intent of the user regarding metric information.",
    )
  
  def metric_discovery(self, agent):
    return Task(description=(f"""
        Answer the general user questions related to discovery, explaination, description, interpretation of metrics/measures/KPIs, tables or columns.
        {self.__tip_section()}"""),
      agent=agent,
      expected_output="If relevant metric exists in metadata, provide the answer. Else, respond with 'Sorry, the query is out of scope.' and terminate process.",
    )
  def __tip_section(self):
    return "If you do your BEST WORK, I'll give you a $10,000 commission!"
  
  

Agents

In [37]:
class MetricDiscoveryAgent():
  def user_intent_capture(self):
    return Agent(
      role='Intent Capturer',
      goal=
      'Rephrasing the query to capture the intent of the user regarding metric information',
      backstory=
      "You are an expert to understand the user's intent and rephrase the query to capture the intent of the user accurately.",
      verbose=True,
      tools=[
        RephraseInputQuery()
      ],
      llm = llm_gpt,
    )
  def discover_metric_info(self):
    return Agent(
     role='Data Analyst Assistant',
      goal=
      'Empower users to understand and utilize data effectively. This includes helping them discover relevant metrics, interpreting their meaning',
      backstory=
      "The primary purpose is to bridge the gap between raw data and user comprehension, fostering a data-driven culture within the organization.",
      llm = llm_gpt,
      verbose=True,
      tools=[
        MetricDiscovery()
      ],
    )

Crew

In [38]:
def after_task_callback(output):
  # Perform actions after the task, 
  # for example, logging or updating agent state
  
  print(f"Agent completed task with result: {output}")

# Assigning the function to task_callback
task_callback = after_task_callback

In [39]:
from crewai.process import Process
class MetricDiscoveryInputCrew:
  def __init__(self, query):
    self.query = query
    # self.llm = llm_gpt

  def run(self):
    agents = MetricDiscoveryAgent()
    tasks = MetricDiscoveryTasks()
    # print(agents)
    print(tasks)
    user_intent_capture_agent = agents.user_intent_capture()
    discover_metric_info_agent = agents.discover_metric_info()
    # print(metric_isolator_agent)
    # metric_isolator_task = tasks.metric_isolation(metric_isolator_agent, self.query)
    metric_isolator_task = tasks.metric_isolation(user_intent_capture_agent, self.query)
    metric_discover_task = tasks.metric_discovery(discover_metric_info_agent)
    
    # print("Metric_isolator_task", metric_isolator_task)
    crew = Crew(
      agents=[
        user_intent_capture_agent,
        discover_metric_info_agent,
      ],
      tasks=[
        metric_isolator_task,
        metric_discover_task
      ],
      verbose=False,
      process=Process.sequential,
      step_callback=after_task_callback(metric_discover_task.output)
    )

    result = crew.kickoff()
    return result

In [40]:
query = "What is the most popular feature used by our paid subscribers"
# formatted_query = input(
#     dedent("""
#       {What is the most popular feature used by our paid subscribers}
#     """))
# print(formatted_query)
crew = MetricDiscoveryInputCrew(query)
result = crew.run()

2024-06-01 10:26:32,139 - 140704506453952 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


Agent completed task with result: None


> Entering new CrewAgentExecutor chain...
Thought: To capture the user's intent regarding metric information accurately, I need to rephrase the query to focus on the metric aspect of the feature usage by paid subscribers.

Action: rephrase_input_query

Action Input: {"query": "What is the most popular feature used by our paid subscribers"}
 

Please provide the feature among our paid subscription offerings that has the highest usage statistics.

Final Answer: Please provide the feature among our paid subscription offerings that has the highest usage statistics.

> Finished chain.


> Entering new CrewAgentExecutor chain...
Thought: To provide the user with the information they're asking for, I need to discover the metric that represents the "highest usage statistics" among our paid subscription features. I will use the metric_discovery tool to find the relevant metric.

Action: metric_discovery

Action Input: {"query": "highest usage statistics 

In [21]:
print(crew.__repr__())